#**Latihan LDA & LSA**
#**Nama : Alya Fauziyah**
#**NIM : 21110030**
#**Kelas : S1-SD02A**

In [5]:
!pip install regex matplotlib sastrawi xlsxwriter nltk

In [6]:
# Import modul
import pandas as pd
import numpy as np
import re as reg
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
data = pd.read_csv('dataset.csv')
data

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1753950154116432276,Sun Feb 04 01:14:38 +0000 2024,13653,PEMBUNUHAN DI SIANG HARI Sebuah kasus pembunuh...,1753950154116432276,https://pbs.twimg.com/media/GFdIXW7aUAAjGuW.jpg,NaN,in,"Sidoarjo, Indonesia",324,170,2292,https://twitter.com/prabu_abimanyu/status/1753...,957568678035308545,prabu_abimanyu
1,1753816569476743417,Sat Feb 03 16:23:49 +0000 2024,203,Loe tau ga knp gerbong tetangga Jor²an keluari...,1753816569476743417,https://pbs.twimg.com/ext_tw_video_thumb/17538...,NaN,in,"Grindelwald, Switzerland",3,8,52,https://twitter.com/PenculikBangsat/status/175...,164370154,PenculikBangsat
2,1753750790475518100,Sat Feb 03 12:03:35 +0000 2024,3736,kasus yg sama tapi di India https://t.co/KscaV...,1753751082814312785,https://pbs.twimg.com/ext_tw_video_thumb/17537...,dxrkchocolx,in,cek likes!,73,33,273,https://twitter.com/dxrkchocolx/status/1753751...,1208278977040642050,dxrkchocolx
3,1752994269181469088,Thu Feb 01 09:56:17 +0000 2024,2585,Yang menjegal Anies akan terjungkal 1. Diroast...,1752994269181469088,https://pbs.twimg.com/ext_tw_video_thumb/17529...,NaN,in,NaN,25,53,1039,https://twitter.com/MichelAdam7__/status/17529...,1647311678160384007,MichelAdam7__
4,1754030008522432851,Sun Feb 04 06:31:56 +0000 2024,16,Ya Allah baru tau kalo ternyata dia yang jadi ...,1754030008522432851,https://pbs.twimg.com/ext_tw_video_thumb/17540...,NaN,in,Selo Kartiko,1,4,7,https://twitter.com/denni_sauya/status/1754030...,1249352392954368001,denni_sauya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,1753185420282892787,Thu Feb 01 22:35:51 +0000 2024,97,Wait.. ini kader PDIP ? Ada nama Cak Imin yg j...,1753185420282892787,NaN,NaN,in,Jakarta Capital Region,1,18,24,https://twitter.com/G4b0nGOKU/status/175318542...,1614470115210457090,G4b0nGOKU
277,1751059387756810249,Sat Jan 27 01:47:45 +0000 2024,0,Why the hell there have been lot of bad news a...,1751059387756810249,NaN,NaN,en,"Tangerang, Indonesia",0,0,0,https://twitter.com/fairdimsum/status/17510593...,3236589794,fairdimsum
278,1751137798474830263,Sat Jan 27 07:15:43 +0000 2024,17,@convefrt HATI HATI YA KALIAN SEMUA BANYAK BGT...,1751141921366712787,NaN,dibeliiii,in,𝙗𝙪𝙨𝙨𝙞𝙣𝙚𝙨𝙨,1,1,1,https://twitter.com/dibeliiii/status/175114192...,1727116330556645376,dibeliiii
279,1752343920489717869,Wed Jan 31 02:53:12 +0000 2024,0,@Fahrihamzah Saya pengen kasus ini d ungkap ht...,1752525408162193617,https://pbs.twimg.com/ext_tw_video_thumb/17525...,Fahrihamzah,in,NaN,0,0,0,https://twitter.com/B36686Brayyy/status/175252...,1728786553613524992,B36686Brayyy


In [8]:
# Preprocessing
import re as reg
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

slangs={'yg':'yang', 'tdk':'tidak', 'pd':'pada', 'mlh':'malah', 'jgn':'jangan', 'jg':'juga', 'tp':'tapi', 'blkg': 'belakang', 'dr':'dari', 'klo':'kalo', 'lg':'lagi', 'btw':'buat',
        'dlm':'dalam','dgn':'dengan', 'poto':'foto', 'g':'tidak', 'n':'dan', 'ad':'ada', 'brp': 'berapa', "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know",
        "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan",
        "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi",
        "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang",
        "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi",
        "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete":
        "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya",
        "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis":
        "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan",
        "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan",
        "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut",
        "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep",
        "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek",
        "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta",
        "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan",
        "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang",
        "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi",
        "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile",
        "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa",
        "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas",
        "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana",
        "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu",
        "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai",
        "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang",
        "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image",
        "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara",
        "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan",
        "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan",
        "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin",
        "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan",
        "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal",
        "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi",
        "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah",
        "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi",
        "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau",
        "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada",
        "kptsan":"keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan",
        "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat",
        "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu",
        "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram",
        "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap",
        "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas",
        "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja",
        "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis",
        "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme",
        "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa",
        "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat",
        "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu",
        "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh",
        "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin",
        "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf",
        "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian",
        "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap",
        "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih",
        "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah",
        "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik",
        "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik",
        "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi",
        "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman",
        "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung",
        "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu",
        "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah",
        "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend",
        "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", 'ga':'tidak', 'cwe':'perempuan', 'dl':'dulu', 'rumayan':'lumayan',
        'ny':'nya', 'htm':'harga tiket masuk', 'cm':'cuma', 'slalu':'selalu', 'tingi':'tinggi','neng':'senang'}
processed_comments = []

for sentence in data['full_text']:
    # Remove all the special characters
    processed_comment = reg.sub(r'\W', ' ', str(sentence))

    # Converting to Lowercase
    processed_comment = processed_comment.lower()

    #Remove number
    processed_comment = reg.sub(r'\d+', ' ', processed_comment)

    # remove all single characters
    processed_comment = reg.sub(r'\s+[a-zA-Z]\s+', ' ', processed_comment)

    #remove duplicate character
    pattern=reg.compile(r"(.)\1{1,}",reg.DOTALL)
    processed_comment=pattern.sub(r"\1",processed_comment)


    #Corrected Slang words
    words = processed_comment.split()
    rfrm=[slangs[word] if word in slangs else word for word in words]
    processed_comment= " ".join(rfrm)

    #remove stopword
    factory = StopWordRemoverFactory()
    more_stopword = ['tak', 'jd', 'per', 'nya', 'terjemah', 'diterjemahkan', 'oleh', 'gogle', 'google' ,'nan', 'baik', 'sangat', 'batas', 'coba',
                        'ada','bersih', 'salur', 'baru', 'purwokerto', 'batas', 'hotel', 'coba', 'putus', 'ada','sama', 'suka', 'bilang',
                        'com', 'kamu', 'http', 'https', 'htps', 'htp', 'gak', 'jadi', 'lebih', 'kalau', 'banyak', 'jangan', 'iya', 'kok',
                        'apa', 'paling', 'semua', 'lah', 'ihwm', 'od', 'pakai', 'hayo', 'no', 'ihwjm', 'od'] #menambahkan stopword
    stopwords = factory.get_stop_words() + more_stopword
    temp = [t for t in reg.findall(r'\b[a-z]+-?[a-z]+\b',processed_comment) if t not in stopwords]
    processed_comment = ' '.join(temp)

    #stemming
    stemmer = StemmerFactory().create_stemmer()
    processed_comment = stemmer.stem(processed_comment)

    # Substituting multiple spaces with single space
    processed_comment = reg.sub(r'\s+', ' ', processed_comment, flags=reg.I)

    processed_comments.append(processed_comment)

# Output data Preprocessing
processed_comments

['bunuh siang hari buah kasus bunuh butuh kolaborasi multidisiplin ilmu pengungkapanya cermat scientific logis co tyq bv cuo',
 'tahu gerbong tetanga keluarin dana kampanye menang yoi nasib ujung tanduk bila menang mungkin ketua kpk amp segala sandera amp pending kupas tuntas pak lurah kasus ganjarmahfud co wr ld',
 'kasus india co kscavrkxug',
 'jegal anies jungkal diroasting giring bahkan populer larang monas bahkan bikin sendiri sirkuit formula background jis lapor kpk kasus formula bahkan dukung luas koalisi bubar bahkan cak imin istanapanik co wplbivymp',
 'alah tahu nyata dalang kasus penistan agama ahok asalbukanprabowo co yof mucep',
 'reka ulang kasus curi emas ponakanya bunvh gila sih jahat banget al fatihah korban co iqh',
 'orang posisi benar takut diancem ancem gertak gertak lagi salah orang mas anies urat takut deh hilang kasus reklamasi teluk jakarta co laglk pt',
 'tahun perintah jokowi janji buka kasus culi hilang paksa aktivis tahun jelas hari bicara isu tahun lihat s

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
processed_comments = ''.join(processed_comments)

In [11]:
type(processed_comments)

str

In [12]:
from nltk.tokenize import word_tokenize
Docs = processed_comments
Sentences = nltk.sent_tokenize(Docs)
print ("Kalimat Awal: ", Sentences)
hasil_tokenizing = nltk.word_tokenize(Docs)
print("Setelah Tokenizing: ", hasil_tokenizing)

Kalimat Awal:  ['bunuh siang hari buah kasus bunuh butuh kolaborasi multidisiplin ilmu pengungkapanya cermat scientific logis co tyq bv cuotahu gerbong tetanga keluarin dana kampanye menang yoi nasib ujung tanduk bila menang mungkin ketua kpk amp segala sandera amp pending kupas tuntas pak lurah kasus ganjarmahfud co wr ldkasus india co kscavrkxugjegal anies jungkal diroasting giring bahkan populer larang monas bahkan bikin sendiri sirkuit formula background jis lapor kpk kasus formula bahkan dukung luas koalisi bubar bahkan cak imin istanapanik co wplbivympalah tahu nyata dalang kasus penistan agama ahok asalbukanprabowo co yof mucepreka ulang kasus curi emas ponakanya bunvh gila sih jahat banget al fatihah korban co iqhorang posisi benar takut diancem ancem gertak gertak lagi salah orang mas anies urat takut deh hilang kasus reklamasi teluk jakarta co laglk pttahun perintah jokowi janji buka kasus culi hilang paksa aktivis tahun jelas hari bicara isu tahun lihat sekali betapa peduli 

In [13]:
len(hasil_tokenizing)

5849

In [14]:
# Ekstraksi fitur / representasi dokumen menggunakan TF
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=1.0, min_df=1)

tf = tf_vectorizer.fit_transform(hasil_tokenizing)

# hasil representasi
tf_terms = tf_vectorizer.get_feature_names_out()
print(tf_vectorizer.get_feature_names_out())
matrix = tf.toarray()
print(matrix)

['abah' 'abai' 'abal' ... 'zul' 'zwk' 'zyb']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## **Menggunakan LDA**

In [15]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 10 #untuk mendapatkan jumlah topik terbaik perlu trial
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0)
lda.fit(tf)
lda

LatentDirichletAllocation(random_state=0)

In [16]:
vsm_topics = lda.transform(tf)
#tampilkan hasil
print(vsm_topics)

[[0.05       0.05       0.05       ... 0.05       0.05       0.05      ]
 [0.55       0.05       0.05       ... 0.05       0.05       0.05      ]
 [0.55       0.05       0.05       ... 0.05       0.05       0.05      ]
 ...
 [0.05       0.05       0.05       ... 0.05       0.05       0.54999999]
 [0.05       0.05       0.05       ... 0.05       0.05       0.05      ]
 [0.05       0.05       0.05       ... 0.05       0.55       0.05      ]]


In [17]:
len(vsm_topics)

5849

In [18]:
#Tampilkan nilai-nilai setiap fitur
print(lda.components_)

[[0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 [2.09999998 0.1        0.1        ... 2.09999998 0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 ...
 [0.1        0.1        2.09999998 ... 0.1        0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        1.09999998]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]]


In [19]:
#hasil label topic model untuk setiap dokumen
import numpy as np
topics = np.argmax(vsm_topics, axis=1)
topics

array([5, 0, 0, ..., 9, 6, 8])

In [20]:
#mencetak word fitur dengan nilai tertinggi pada setiap topik
n_top_words = 10 # jumlah fitur tertinggi yang kita tentukan
topic_words = {}
for topic, comp in enumerate(lda.components_):
 word_idx = np.argsort(comp)[::-1][:n_top_words] # susun indeks secara terbalik, sehingga yang paling tinggi muncul di awal.
 # store the words most relevant to the topic
 topic_words[topic] = [tf_vectorizer.get_feature_names_out()[i]+' '+str(comp[i]) for i in word_idx]

In [21]:
type(topic_words)

dict

In [22]:
topic_words

{0: ['kasus 281.0999999858961',
  'pak 23.09999998566645',
  'cuma 17.099999985577362',
  'the 12.099999985433392',
  'pernah 11.099999985388475',
  'kalian 11.099999985388475',
  'hilang 10.099999985334316',
  'selesai 10.099999985334316',
  'hari 9.099999985267738',
  'penistan 9.099999985267738'],
 1: ['buat 26.099999987148603',
  'banget 20.09999998708864',
  'korban 19.099999987074927',
  'salah 16.099999987023256',
  'terus 13.099999986947202',
  'bikin 13.099999986947202',
  'pilih 11.099999986872838',
  'masa 11.099999986872838',
  'jelas 10.099999986824182',
  'jokowi 9.099999986764367'],
 2: ['uang 20.099999988433783',
  'libat 20.099999988433783',
  'dukung 18.099999988407795',
  'lapor 12.099999988276604',
  'dulu 10.09999998819687',
  'bagaimana 10.099999988196869',
  'aman 9.099999988143285',
  'lalu 8.099999988075822',
  'kan 8.099999988075822',
  'sekali 8.099999988075822'],
 3: ['korupsi 27.099999988151417',
  'amp 20.099999988089294',
  'mau 19.099999988076632',
  'ta

In [23]:
for topic, words in topic_words.items():
 print('Topic: %d' % topic)
 print(' %s' % ', '.join(words))

Topic: 0
 kasus 281.0999999858961, pak 23.09999998566645, cuma 17.099999985577362, the 12.099999985433392, pernah 11.099999985388475, kalian 11.099999985388475, hilang 10.099999985334316, selesai 10.099999985334316, hari 9.099999985267738, penistan 9.099999985267738
Topic: 1
 buat 26.099999987148603, banget 20.09999998708864, korban 19.099999987074927, salah 16.099999987023256, terus 13.099999986947202, bikin 13.099999986947202, pilih 11.099999986872838, masa 11.099999986872838, jelas 10.099999986824182, jokowi 9.099999986764367
Topic: 2
 uang 20.099999988433783, libat 20.099999988433783, dukung 18.099999988407795, lapor 12.099999988276604, dulu 10.09999998819687, bagaimana 10.099999988196869, aman 9.099999988143285, lalu 8.099999988075822, kan 8.099999988075822, sekali 8.099999988075822
Topic: 3
 korupsi 27.099999988151417, amp 20.099999988089294, mau 19.099999988076632, tahun 14.09999998798561, deh 13.09999998795881, presiden 11.099999987890214, agama 10.099999987845328, gara 10.0999

**Interpretasi hasil :**

Misal topic 0, didalamnya terdapat kata kasus, pak, cuma, the, pernah, dsb dengan probabilitas masing - masing. Di mana kita dapat melihat bahwa kata kasus memiliki nilai 281.09. Artinya bahwa kata "kasus" dalam topik tersebut memiliki distrusi probalitas paling tinggi.

## **Menggunakan LSA**

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vektor = TfidfVectorizer(max_features=400)

In [25]:
df = pd.read_csv('dataset.csv')

In [26]:
df.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1753950154116432276,Sun Feb 04 01:14:38 +0000 2024,13653,PEMBUNUHAN DI SIANG HARI Sebuah kasus pembunuh...,1753950154116432276,https://pbs.twimg.com/media/GFdIXW7aUAAjGuW.jpg,NaN,in,"Sidoarjo, Indonesia",324,170,2292,https://twitter.com/prabu_abimanyu/status/1753...,957568678035308545,prabu_abimanyu
1,1753816569476743417,Sat Feb 03 16:23:49 +0000 2024,203,Loe tau ga knp gerbong tetangga Jor²an keluari...,1753816569476743417,https://pbs.twimg.com/ext_tw_video_thumb/17538...,NaN,in,"Grindelwald, Switzerland",3,8,52,https://twitter.com/PenculikBangsat/status/175...,164370154,PenculikBangsat
2,1753750790475518100,Sat Feb 03 12:03:35 +0000 2024,3736,kasus yg sama tapi di India https://t.co/KscaV...,1753751082814312785,https://pbs.twimg.com/ext_tw_video_thumb/17537...,dxrkchocolx,in,cek likes!,73,33,273,https://twitter.com/dxrkchocolx/status/1753751...,1208278977040642050,dxrkchocolx
3,1752994269181469088,Thu Feb 01 09:56:17 +0000 2024,2585,Yang menjegal Anies akan terjungkal 1. Diroast...,1752994269181469088,https://pbs.twimg.com/ext_tw_video_thumb/17529...,NaN,in,NaN,25,53,1039,https://twitter.com/MichelAdam7__/status/17529...,1647311678160384007,MichelAdam7__
4,1754030008522432851,Sun Feb 04 06:31:56 +0000 2024,16,Ya Allah baru tau kalo ternyata dia yang jadi ...,1754030008522432851,https://pbs.twimg.com/ext_tw_video_thumb/17540...,NaN,in,Selo Kartiko,1,4,7,https://twitter.com/denni_sauya/status/1754030...,1249352392954368001,denni_sauya


In [27]:
df = df['full_text']

In [28]:
df = pd.DataFrame(df)

In [29]:
df.head()

,full_text
0,PEMBUNUHAN DI SIANG HARI Sebuah kasus pembunuh...
1,Loe tau ga knp gerbong tetangga Jor²an keluari...
2,kasus yg sama tapi di India https://t.co/KscaV...
3,Yang menjegal Anies akan terjungkal 1. Diroast...
4,Ya Allah baru tau kalo ternyata dia yang jadi ...


In [30]:
#menghitung tf-idf dengan TfidfTransformer
vektor_dt=vektor.fit_transform(df['full_text'].values.astype('U'))
print (vektor_dt)
print (vektor_dt.shape)

  (0, 78)	0.11101321081239572
  (0, 134)	0.11101321081239572
  (0, 84)	0.17782595561800688
  (0, 82)	0.2952016139310396
  (0, 395)	0.19178005258049574
  (0, 170)	0.08269502508164485
  (0, 129)	0.32080852145064936
  (0, 91)	0.16105803105574315
  (0, 276)	0.8271095563148579
  (1, 251)	0.19157623756026299
  (1, 9)	0.13941084785366942
  (1, 261)	0.19437761260102351
  (1, 17)	0.19157623756026299
  (1, 328)	0.18398029858206205
  (1, 364)	0.24331282909268118
  (1, 25)	0.37783717497083386
  (1, 196)	0.19157623756026299
  (1, 188)	0.24331282909268118
  (1, 150)	0.26074640351049094
  (1, 236)	0.20382420605627072
  (1, 235)	0.5214928070209819
  (1, 165)	0.23029152061356048
  (1, 115)	0.19157623756026299
  (1, 352)	0.20382420605627072
  (1, 78)	0.07309744210927126
  :	:
  (278, 174)	0.16330943004900883
  (278, 73)	0.2291944083222713
  (278, 68)	0.18045987373524858
  (278, 162)	0.21692867164252955
  (278, 92)	0.18309869675724108
  (278, 394)	0.15501728040312862
  (278, 396)	0.11153215562152466
  (2

In [31]:
# topic modeling using LSA
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=5, n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vektor_dt)
print(lsa_top)

[[ 0.24489392 -0.03299483  0.04468685  0.11017428 -0.0706721 ]
 [ 0.2574406   0.06202824 -0.00622217 -0.07019275 -0.05792675]
 [ 0.43916557 -0.03676021 -0.01696484 -0.20779744 -0.04608295]
 ...
 [ 0.1863004   0.09574083 -0.17533301 -0.10586382  0.0529373 ]
 [ 0.29529912 -0.13863177  0.37550745  0.20018226  0.1873339 ]
 [ 0.21061474 -0.0542782   0.16932642 -0.08459686 -0.07850294]]


In [32]:
len(lsa_top)

281

In [33]:
lsa_model.get_params()

{'algorithm': 'randomized',
 'n_components': 5,
 'n_iter': 10,
 'n_oversamples': 10,
 'power_iteration_normalizer': 'auto',
 'random_state': 42,
 'tol': 0.0}

In [34]:
# Memunculkan nilai lsa setiap topik
l=lsa_top[0]
print("Topik- Topik:")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Topik- Topik:
Topic  0  :  24.48939205115884
Topic  1  :  -3.299483099895188
Topic  2  :  4.4686850962989855
Topic  3  :  11.01742761619714
Topic  4  :  -7.06721042520225


In [35]:
# Memunculkan jumlah kata-kata dalam setiap topik
print(lsa_model.components_.shape)
print(lsa_model.components_)

(5, 400)
[[ 1.85015241e-02  5.88822759e-02  2.43492109e-02 ...  1.56573564e-02
   6.32247140e-03  2.22711349e-02]
 [-1.30119150e-02 -2.89562226e-02 -6.58324558e-04 ... -9.65710952e-03
  -4.29626584e-03 -2.72892895e-02]
 [-3.32844689e-03  8.10170347e-03  2.46733987e-02 ... -2.77526211e-04
  -1.32632987e-02 -8.11958373e-02]
 [-1.18225738e-02 -1.02181715e-01  3.73236062e-02 ...  2.54792279e-02
   2.36031839e-03 -4.65885077e-02]
 [-2.42424169e-02 -6.54073501e-02 -1.16090003e-02 ...  2.10883943e-02
   6.58769776e-03  3.00338064e-01]]


In [37]:
# Word/ kata paling penting dalam setiap topik
vocab = vektor.get_feature_names_out()
for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=", ")
    print("\n")

Topic 0: 
co, https, kasus, yg, ini, di, dan, yang, ada, bisa, 

Topic 1: 
tim, agama, dalang, penistaan, ternyata, ahok, prabowo, allah, tau, baru, 

Topic 2: 
co, https, fahrihamzah, hamzah, fahri, benur, kasus, saya, dugaan, takutnya, 

Topic 3: 
yang, ini, saya, korban, dan, dengan, tidak, buat, hari, fahrihamzah, 

Topic 4: 
joki, zonauang, tugas, fahrihamzah, jokitugas, hamzah, ini, fahri, benur, sebut, 



In [38]:
for topic, words in topic_words.items():
 print('Topic: %d' % topic)
 print(' %s' % ', '.join(words))

Topic: 0
 kasus 281.0999999858961, pak 23.09999998566645, cuma 17.099999985577362, the 12.099999985433392, pernah 11.099999985388475, kalian 11.099999985388475, hilang 10.099999985334316, selesai 10.099999985334316, hari 9.099999985267738, penistan 9.099999985267738
Topic: 1
 buat 26.099999987148603, banget 20.09999998708864, korban 19.099999987074927, salah 16.099999987023256, terus 13.099999986947202, bikin 13.099999986947202, pilih 11.099999986872838, masa 11.099999986872838, jelas 10.099999986824182, jokowi 9.099999986764367
Topic: 2
 uang 20.099999988433783, libat 20.099999988433783, dukung 18.099999988407795, lapor 12.099999988276604, dulu 10.09999998819687, bagaimana 10.099999988196869, aman 9.099999988143285, lalu 8.099999988075822, kan 8.099999988075822, sekali 8.099999988075822
Topic: 3
 korupsi 27.099999988151417, amp 20.099999988089294, mau 19.099999988076632, tahun 14.09999998798561, deh 13.09999998795881, presiden 11.099999987890214, agama 10.099999987845328, gara 10.0999

**Interpretasi hasil :**

Misal topic 0, didalamnya terdapat kata kasus, pak, cuma, the, pernah, dsb dengan probabilitas masing - masing. Di mana kita dapat melihat bahwa kata kasus memiliki nilai 281.09. Artinya bahwa kata "kasus" dalam topik tersebut memiliki distrusi probalitas paling tinggi.